# CLINICAL TRIAL ANALYTICAL DASHBOARD

## Before to start


Befor to start, ensure the kernel used to run this Notebook have installed the proper packages. I usually work with uv package manager and create specific Kernels, I proceed with following code to test package availability & installation if required:

In [63]:
!uv pip freeze | grep mysql-connector-python || uv add mysql-connector-python

mysql-connector-python==9.5.0


In [62]:
!uv pip freeze | grep tqdm || uv add tqdm

tqdm==4.67.2


In [66]:
!uv pip freeze | grep schedule || uv add schedule

Resolved 135 packages in 808ms                                       
⠙ Preparing packages... (0/1)                                                   ⠋ Preparing packages... (0/0)                                                   
⠙ Preparing packages... (0/1)-------------------     0 B/11.93 KiB           
⠙ Preparing packages... (0/1)---------- 11.93 KiB/11.93 KiB         
Prepared 1 package in 53ms                                                        
Installed 1 package in 1ms                                  
 + schedule==1.2.2


In [80]:
!uv pip freeze | grep plotly || uv add plotly

plotly==6.5.2


## 1. IMPORTS


In [42]:
import pandas as pd
import mysql.connector
from mysql.connector import Error
import os
from dotenv import load_dotenv
from pathlib import Path
import plotly.express as px
import seaborn as sns
from matplotlib import pyplot as plt
from loguru import logger
# Siempre usa display() en lugar de print()
from IPython.display import display


## 2. FUNCTIONS


In [14]:

# Database connection with retry for the seed check
def get_db_connection_with_retry(retries=3, delay=1):
    for attempt in range(retries):
        try:
            connection = mysql.connector.connect(
                host=os.getenv('DB_HOST', 'mysql'),
                port=int(os.getenv('DB_PORT', 3306)),
                user=os.getenv('DB_USER', 'user'),
                password=os.getenv('DB_PASSWORD', 'pass'),
                database=os.getenv('DB_NAME', 'clinicaltrials')
            )
            return connection
        except Error as e:
            if attempt < retries - 1:
                time.sleep(delay)
            else:
                logger.error(f"Database connection failed after {retries} attempts: {e}")
                return None
    return None


In [50]:
def table_summary(df_dict, table_name):
    df = df_dict[table_name]
    summary = pd.DataFrame({
        'Column': df.columns,
        'Total_Values': df.count(),
        'Different_Values': df.nunique(),
        'Missing_Count': df.isnull().sum(),
        'Missing_%': (df.isnull().sum() / len(df) * 100).round(1),
        'Data_Type': df.dtypes
    })
    summary['Unique_%'] = (summary['Different_Values'] / summary['Total_Values'] * 100).round(1)
    return summary.sort_values('Missing_%', ascending=False).reset_index(drop=True)


In [79]:
def count_empty_strings(df):
    """
    Cuenta cadenas vacías ("") en TODAS las columnas del DataFrame.
    
    Args:
        df (pd.DataFrame): DataFrame a analizar
    
    Returns:
        pd.DataFrame: Reporte con conteos y % de "" por columna
    """
    import pandas as pd
    
    # Detecta "" en todas las columnas
    empty_counts = {}
    
    for col in df.columns:
        # Cuenta exactamente "" (no NaN, no espacios)
        empty_mask = (df[col].astype(str) == "")  # Convierte a str para uniformidad
        count = empty_mask.sum()
        total = len(df)
        pct = (count / total * 100).round(2)
        
        empty_counts[col] = {
            'Empty_Strings': count,
            'Total_Rows': total,
            'Empty_%': pct
        }
    
    # Reporte ordenado por más "" primero
    report = pd.DataFrame(empty_counts).T
    report = report.sort_values('Empty_%', ascending=False).reset_index(drop=True)
    
    print(f"🔍 ANÁLISIS CADENAS VACÍAS ('') - Shape: {df.shape}")
    print("=" * 60)
    
    if report['Empty_Strings'].sum() == 0:
        print("✅ NO se encontraron cadenas vacías ('') en ninguna columna")
    else:
        display(report)
        print(f"\n📊 TOTAL cadenas vacías: {report['Empty_Strings'].sum()}")
    
    return report



In [ ]:
# NOT USED IN THIS NOTEBOOK! ONLY FOR REFERENCE

def get_mesh_term(condition_name: str) -> Optional[str]:
    """Fetch MeSH descriptor ID for a condition name."""
    if not condition_name:
        return None

    url = "https://id.nlm.nih.gov/mesh/lookup/descriptor"
    params = {
        "label": condition_name,
        "match": "contains",
    }

    try:
        resp = requests.get(url, params=params, timeout=10)
        resp.raise_for_status()
        results = resp.json() or []
        if not results:
            return None
        resource = results[0].get("resource")
        if not resource:
            return None
        return resource.rsplit("/", 1)[-1]
    except Exception as e:
        logger.error(f"error:{e}")
        return None


## 3. DDBB CONECTION & DATAFRAMES GENERATION


In case you can not access to ddbb due to run it through a jupyter notebook. You can proceed with the next steps to grant acces to ddbb:
1. Get inside the mysql docker to modify privileges (use the root's passord)
```bash
docker exec -it mysql-clinical-db  mysql -u root -p
```
2.  Inside MySQL (modify the tu_password to the root's passord):
```sql
GRANT ALL PRIVILEGES ON *.* TO 'root'@'%' IDENTIFIED BY 'tu_password';
GRANT ALL PRIVILEGES ON *.* TO 'root'@'172.%' IDENTIFIED BY 'tu_password';  -- Cubre Docker IPs
FLUSH PRIVILEGES;
SELECT user, host FROM mysql.user WHERE user='root';  -- Verifica '%'
EXIT;
```
It should print on screen:
```
+------+-----------+
| user | host      |
+------+-----------+
| root | %         |
| root | localhost |
+------+-----------+
```


3.  Restart dockers:
```bash
docker compose down && docker compose up -d
```

4.  Connect to ddbb using root as user:

        os.environ['DB_HOST'] = '127.0.0.1' # e.g., 'your_remote_db_host.com' or '127.0.0.1' if running locally

        os.environ['DB_PORT'] = '3306' # Your specific MySQL port

        os.environ['DB_USER'] = 'root' # Your MySQL username

        os.environ['DB_PASSWORD'] = 'rootpass' # Your MySQL password

        os.environ['DB_NAME'] = 'clinicaltrials' # Your database name





In [ ]:
# --- IMPORTANT: Replace these with your actual MySQL database credentials ---
# Ensure your MySQL database is accessible from this Colab environment.


# Uncomment the following two lines if you have a .env file with your credentials
# from dotenv import load_dotenv
# load_dotenv()

# Alternatively, set them directly as environment variables:
# Example placeholders. You MUST change these to your actual database details.
os.environ['DB_HOST'] = '127.0.0.1' # e.g., 'your_remote_db_host.com' or '127.0.0.1' if running locally
os.environ['DB_PORT'] = '3306' # Your specific MySQL port, often 3306
os.environ['DB_USER'] = 'root' # Your MySQL username
os.environ['DB_PASSWORD'] = 'rootpass' # Your MySQL password
os.environ['DB_NAME'] = 'clinicaltrials' # Your database name

print("Placeholder MySQL database credentials have been set. Please update them with your actual details.")
print("After updating, re-run the connection cell below (J1qAT_SARyIP).")

Placeholder MySQL database credentials have been set. Please update them with your actual details.
After updating, re-run the connection cell below (J1qAT_SARyIP).


In [ ]:
tables_ddbb= ['studies', 'conditions', 'interventions', 'outcomes', 'sponsors', 'locations', 'study_design']


In [ ]:
connection = get_db_connection_with_retry()
dict_df_clinical_trials = {}
if connection:
    try:

        for table in tables_ddbb:
            query = f'SELECT * FROM {table}'
            dict_df_clinical_trials[table] = pd.read_sql_query(query, connection)
            logger.success(f"Data from '{table}' table loaded successfully. Number of records: {len(dict_df_clinical_trials[table])}")
        logger.success("All Data from ddbb loaded successfully.")
    except Error as e:
        logger.error(f"Error reading data from MySQL: {e}")
    finally:
        if connection.is_connected():
            connection.close()
            logger.info("MySQL connection closed.")
else:
    logger.error("Failed to connect to MySQL database.")

/tmp/ipykernel_276868/550113246.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dict_df_clinical_trials[table] = pd.read_sql_query(query, connection)
2026-02-03 09:40:55.150 | SUCCESS  | __main__:<module>:9 - Data from 'studies' table loaded successfully. Number of records: 10000
2026-02-03 09:40:55.188 | SUCCESS  | __main__:<module>:9 - Data from 'conditions' table loaded successfully. Number of records: 17750
2026-02-03 09:40:55.234 | SUCCESS  | __main__:<module>:9 - Data from 'interventions' table loaded successfully. Number of records: 16715
2026-02-03 09:40:55.416 | SUCCESS  | __main__:<module>:9 - Data from 'outcomes' table loaded successfully. Number of records: 61766
2026-02-03 09:40:55.456 | SUCCESS  | __main__:<module>:9 - Data from 'sponsors' table loaded successfully. Number of records: 15861
2026-02-03 09:40:55.615

## 4. DATA EXPLORATORY ANALYSIS

### 4.1. Statistical Analysis

In [27]:
for table in tables_ddbb:
    print (f"# # # # # # # {table}   # # # # # # #")
    display(dict_df_clinical_trials[table].describe())
            

# # # # # # # studies   # # # # # # #


,study_id,enrollment,created_at,updated_at
count,10000.00000,9.872000e+03,10000,10000
mean,5000.50000,2.066886e+03,2026-02-02 19:29:38.556600,2026-02-02 19:29:38.556600
min,1.00000,0.000000e+00,2026-02-02 19:29:25,2026-02-02 19:29:25
25%,2500.75000,3.000000e+01,2026-02-02 19:29:32,2026-02-02 19:29:32
50%,5000.50000,6.800000e+01,2026-02-02 19:29:38,2026-02-02 19:29:38
75%,7500.25000,1.900000e+02,2026-02-02 19:29:46,2026-02-02 19:29:46
max,10000.00000,4.238504e+06,2026-02-02 19:29:52,2026-02-02 19:29:52
std,2886.89568,5.925177e+04,NaN,NaN


# # # # # # # conditions   # # # # # # #


,condition_id,study_id
count,17750.000000,17750.000000
mean,8875.500000,4962.439211
std,5124.127975,2875.612055
min,1.000000,1.000000
25%,4438.250000,2485.000000
50%,8875.500000,4972.000000
75%,13312.750000,7408.000000
max,17750.000000,10000.000000


# # # # # # # interventions   # # # # # # #


...

,sponsor_id,study_id
count,15861.000000,15861.000000
mean,7931.000000,4972.140975
std,4578.820645,2904.195303
min,1.000000,1.000000
25%,3966.000000,2461.000000
50%,7931.000000,5000.000000
75%,11896.000000,7460.000000
max,15861.000000,10000.000000


# # # # # # # locations   # # # # # # #


,location_id,study_id
count,56916.000000,56916.000000
mean,28458.500000,5297.656406
std,16430.378298,2936.649709
min,1.000000,1.000000
25%,14229.750000,2851.000000
50%,28458.500000,5353.000000
75%,42687.250000,7848.000000
max,56916.000000,10000.000000


# # # # # # # study_design   # # # # # # #


,design_id,study_id
count,9851.000000,9851.000000
mean,4926.000000,5008.067303
std,2843.883085,2888.100171
min,1.000000,1.000000
25%,2463.500000,2507.500000
50%,4926.000000,5016.000000
75%,7388.500000,7509.500000
max,9851.000000,10000.000000


The only true numerical column is number of enrollments. Such variable has a great distribution with a range of 

In [40]:
for table in tables_ddbb:
    print (f"# # # # # # {table} ({len(dict_df_clinical_trials[table])} records in total) # # # # # # ")
    print (f"Missing values:")
    df=pd.DataFrame(dict_df_clinical_trials[table].isna().sum(), columns=['Missing Values'])
    df['% missing'] = ((df['Missing Values'] / len(dict_df_clinical_trials[table])) * 100).round(2)

    df['type'] = dict_df_clinical_trials[table].dtypes
    display(df)
       

# # # # # # studies (10000 records in total) # # # # # # 
Missing values:


,Missing Values,% missing,type
study_id,0,0.00,int64
nct_id,0,0.00,str
title,0,0.00,str
acronym,7188,71.88,str
status,0,0.00,str
phase,2455,24.55,str
study_type,0,0.00,str
start_date,103,1.03,object
completion_date,285,2.85,object
primary_completion_date,382,3.82,object


# # # # # # conditions (17750 records in total) # # # # # # 
Missing values:


,Missing Values,% missing,type
condition_id,0,0.0,int64
study_id,0,0.0,int64
condition_name,0,0.0,str
mesh_term,17750,100.0,object


# # # # # # interventions (16715 records in total) # # # # # # 
Missing values:


...

,Missing Values,% missing,type
sponsor_id,0,0.00,int64
study_id,0,0.00,int64
agency,0,0.00,str
agency_class,23,0.15,str
lead_or_collaborator,0,0.00,str


# # # # # # locations (56916 records in total) # # # # # # 
Missing values:


,Missing Values,% missing,type
location_id,0,0.00,int64
study_id,0,0.00,int64
facility,0,0.00,str
city,0,0.00,str
state,0,0.00,str
country,0,0.00,str
continent,20,0.04,str


# # # # # # study_design (9851 records in total) # # # # # # 
Missing values:


,Missing Values,% missing,type
design_id,0,0.00,int64
study_id,0,0.00,int64
allocation,2397,24.33,str
intervention_model,2405,24.41,str
masking,2396,24.32,str
primary_purpose,2431,24.68,str
observational_model,7540,76.54,str


### 4.2. Missing data & Uniqueness analysis

In [51]:
for table in tables_ddbb:
    print (f"# # # # # # {table} ({len(dict_df_clinical_trials[table])} records in total) # # # # # # ")
    df_summary =table_summary(df_dict=dict_df_clinical_trials, table_name=table)
    # print (f"Distribution:")
    # df=pd.DataFrame(dict_df_clinical_trials[table].count(), columns=['Total Values Values'])
    # df['unic values'] = df['Different Values'].unique().sum() 
    display(df_summary)
       

# # # # # # studies (10000 records in total) # # # # # # 


,Column,Total_Values,Different_Values,Missing_Count,Missing_%,Data_Type,Unique_%
0,acronym,2812,2758,7188,71.9,str,98.1
1,maximum_age,5367,176,4633,46.3,str,3.3
2,phase,7545,6,2455,24.6,str,0.1
3,minimum_age,9352,121,648,6.5,str,1.3
4,primary_completion_date,9618,2620,382,3.8,object,27.2
5,enrollment_type,9689,2,311,3.1,str,0.0
6,completion_date,9715,2638,285,2.8,object,27.2
7,enrollment,9872,1090,128,1.3,float64,11.0
8,start_date,9897,2916,103,1.0,object,29.5
9,study_id,10000,10000,0,0.0,int64,100.0


# # # # # # conditions (17750 records in total) # # # # # # 


,Column,Total_Values,Different_Values,Missing_Count,Missing_%,Data_Type,Unique_%
0,mesh_term,0,0,17750,100.0,object,NaN
1,condition_id,17750,17750,0,0.0,int64,100.0
2,study_id,17750,9977,0,0.0,int64,56.2
3,condition_name,17750,8850,0,0.0,str,49.9


# # # # # # interventions (16715 records in total) # # # # # # 


...

,Column,Total_Values,Different_Values,Missing_Count,Missing_%,Data_Type,Unique_%
0,agency_class,15838,8,23,0.1,str,0.1
1,sponsor_id,15861,15861,0,0.0,int64,100.0
2,study_id,15861,10000,0,0.0,int64,63.0
3,agency,15861,6621,0,0.0,str,41.7
4,lead_or_collaborator,15861,2,0,0.0,str,0.0


# # # # # # locations (56916 records in total) # # # # # # 


,Column,Total_Values,Different_Values,Missing_Count,Missing_%,Data_Type,Unique_%
0,location_id,56916,56916,0,0.0,int64,100.0
1,study_id,56916,9000,0,0.0,int64,15.8
2,facility,56916,29877,0,0.0,str,52.5
3,city,56916,6553,0,0.0,str,11.5
4,state,56916,1660,0,0.0,str,2.9
5,country,56916,149,0,0.0,str,0.3
6,continent,56896,6,20,0.0,str,0.0


# # # # # # study_design (9851 records in total) # # # # # # 


,Column,Total_Values,Different_Values,Missing_Count,Missing_%,Data_Type,Unique_%
0,observational_model,2311,9,7540,76.5,str,0.4
1,primary_purpose,7420,9,2431,24.7,str,0.1
2,intervention_model,7446,5,2405,24.4,str,0.1
3,allocation,7454,3,2397,24.3,str,0.0
4,masking,7455,5,2396,24.3,str,0.1
5,study_id,9851,9851,0,0.0,int64,100.0
6,design_id,9851,9851,0,0.0,int64,100.0


***Coments***: 
- Column **continent** was filled using data from country's column and packages pycountry (done when modifying ddbb with )
- **Mesh_term** data was filled when available using an external API ("https://id.nlm.nih.gov/mesh/lookup/descriptor") and data from condition_name's column.  (done when modifying ddbb with )
- Column *mesh_term* was removed as no clear way to get its value (to be reevaluted)
- Unique_% in certain cases shows warnings of duplicity (for instance in "acronym", "title", "brief_summary" columns in "studies" table, values near 100% but lower indicate duplicity) --> to be checked!
- On the other side, unique_% close to 0 values indicates low --> to be depeer studied!

### 4.3. Duplicity & Distribution analysis

In [75]:
for table in tables_ddbb:
    print (f"# # # # # # {table} ({len(dict_df_clinical_trials[table])} records in total) # # # # # # ")
    for col in dict_df_clinical_trials[table].columns:
        if "_id" not in col:
            vc = pd.DataFrame(dict_df_clinical_trials[table][col].value_counts().head(5))
            display(vc) 

# # # # # # studies (10000 records in total) # # # # # # 


,count
title,
[Trial of device that is not approved or cleared by the U.S. FDA],23
Efficacy and Safety Study of SHP647 as Induction Therapy in Participants With Moderate to Severe Ulcerative Colitis,2
"Quantitative and Clinical Assessment of Flexor Tendon Gliding Following Application of a Bioresorbable Hydrogel: A Prospective, Randomized Study in Patients Undergoing Distal Radius Fracture Repair",1
Functional Outcomes After Nerve Sparing Surgery for Deep Endometriosis,1
Alcohol Screening and Preoperative Intervention Research Study - 2,1


,count
acronym,
RCT,5
SMART,4
PREDICT,3
SCOPE,3
SAFE,3


,count
status,
COMPLETED,5406
UNKNOWN,1566
RECRUITING,1155
TERMINATED,616
NOT_YET_RECRUITING,440


,count
phase,
NA,3773
PHASE2,1217
PHASE1,1127
PHASE3,696
PHASE4,631


...

,count
allocation,
RANDOMIZED,4955
NA,1673
NON_RANDOMIZED,826


,count
intervention_model,
PARALLEL,4492
SINGLE_GROUP,2026
CROSSOVER,591
SEQUENTIAL,234
FACTORIAL,103


,count
masking,
NONE,4139
SINGLE,1134
DOUBLE,1035
QUADRUPLE,634
TRIPLE,513


,count
primary_purpose,
TREATMENT,4787
PREVENTION,816
SUPPORTIVE_CARE,428
OTHER,424
BASIC_SCIENCE,351


,count
observational_model,
COHORT,1430
CASE_ONLY,301
CASE_CONTROL,280
OTHER,239
ECOLOGIC_OR_COMMUNITY,26


Data duplicity for a title of one study was considered as a warning signal, let's deep in this case

In [74]:
dict_df_clinical_trials[table][dict_df_clinical_trials["studies"]["title"]=="Efficacy and Safety Study of SHP647 as Induction Therapy in Participants With Moderate to Severe Ulcerative Colitis"]

/tmp/ipykernel_276868/484043594.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dict_df_clinical_trials[table][dict_df_clinical_trials["studies"]["title"]=="Efficacy and Safety Study of SHP647 as Induction Therapy in Participants With Moderate to Severe Ulcerative Colitis"]


,design_id,study_id,allocation,intervention_model,masking,primary_purpose,observational_model
3981,3982,4057,RANDOMIZED,CROSSOVER,NONE,OTHER,NaN
6839,6840,6955,NA,SINGLE_GROUP,NONE,TREATMENT,NaN


Actually, the 2 studies shares the title but have a different design. It's not a relevant issue.

In [ ]:
count_empty_strings(df)

Comments:
- Same title was found for 2 different studies, let's check it: in the next point.
- Generic title "[Trial of device that is not approved or cleared by the U.S. FDA]" was used 23 times 

Warnings:
- values "" are counted as a value instead of missing values (see "state", "facility",...) --> to be checked if there is relevance on missing value determination for such values
- in "measure" from table "outcomes" 4 different options that should be the same due to no difference in reporting Overall survival 107 / Overall Survival (OS)	89 / Overall Survival	79 / Overall survival (OS)74  --> to be deeper studied if info considered relevant!


In [ ]:
dict_df_clinical_trials[table][dict_df_clinical_trials["studies"]["title"]=="Efficacy and Safety Study of SHP647 as Induction Therapy in Participants With Moderate to Severe Ulcerative Colitis"]

## ??. DATA QUERIES (SQL)
Next text is generated by AI when asking to group all sql queries performed in the clinical dashboard platform

SQL Queries Extracted

Query: SELECT COUNT(*) as total FROM studies

Counts total studies; used to detect if DB is empty (0 rows → empty).​
Row Counts

Query: SELECT COUNT(*) as count FROM {selected_table} (dynamic: studies, conditions, etc.)

Fetches row count for any selected table; displays as metric.​
Describe Tables

Query: DESCRIBE {selected_table}

Lists columns/types for data availability analysis.​
Data Availability

Query: SELECT COUNT(*) as total, SUM(CASE WHEN {col} IS NULL THEN 1 ELSE 0 END) as null_count FROM {selected_table}

Per-column null/total counts → availability % (e.g., non-null/total*100).​
Complete Records

Query: SELECT COUNT(*) as complete_count FROM {selected_table} WHERE {col1} IS NOT NULL AND {col2} IS NOT NULL ...

Counts rows with NO nulls across all columns.​
Summary Stats

Query: SELECT COUNT(*) as total, COUNT(DISTINCT {col}) as distinct_count FROM {selected_table} WHERE {col} IS NOT NULL

Total vs unique non-null values per key column (status, phase, etc.).​
Top Values

Query: SELECT {col}, COUNT(*) as count FROM {selected_table} WHERE {col}IS NOT NULL GROUP BY{col} ORDER BY count DESC LIMIT 5

Top 5 frequent values per column (e.g., status, condition_name).​
Studies Distributions

Queries:

    SELECT status, COUNT(*) as count FROM studies WHERE status IS NOT NULL GROUP BY status ORDER BY count DESC

    SELECT phase, COUNT(*) as count FROM studies WHERE phase IS NOT NULL GROUP BY phase ORDER BY count DESC

    SELECT gender, COUNT(*) as count FROM studies WHERE gender IS NOT NULL GROUP BY gender ORDER BY count DESC

Aggregates for bar/pie charts on status, phase, gender.​
Enrollment Stats

Queries:

    SELECT enrollment FROM studies WHERE enrollment IS NOT NULL (for histogram)

    SELECT AVG(enrollment) as avg_enrollment, MIN(enrollment) as min, MAX(enrollment) as max, COUNT(*) as total_studies FROM studies WHERE enrollment IS NOT NULL

    SELECT COUNT(*) as zero_enrollment FROM studies WHERE enrollment = 0

Stats (avg/min/max/zeros) for enrollment metrics/histogram.​
Top Entities

Queries:

    SELECT condition_name, COUNT(*) as count FROM conditions GROUP BY condition_name ORDER BY count DESC LIMIT 15

    SELECT intervention_type, COUNT(*) as count FROM interventions WHERE intervention_type IS NOT NULL GROUP BY intervention_type ORDER BY count DESC

    SELECT outcome_type, COUNT(*) as count FROM outcomes WHERE outcome_type IS NOT NULL GROUP BY outcome_type ORDER BY count DESC

    SELECT agency, COUNT(*) as count FROM sponsors GROUP BY agency ORDER BY count DESC LIMIT 15

    SELECT country, COUNT(*) as count FROM locations WHERE country IS NOT NULL GROUP BY country ORDER BY count DESC LIMIT 15

    SELECT allocation, COUNT(*) as count FROM study_design WHERE allocation IS NOT NULL GROUP BY allocation ORDER BY count DESC

    SELECT primary_purpose, COUNT(*) as count FROM study_design WHERE primary_purpose IS NOT NULL GROUP BY primary_purpose ORDER BY count DESC

Top-N (5-15) by frequency for viz (bars, lollipops).​
Time Trends

Queries:

    SELECT YEAR(start_date) as year, COUNT(*) as study_count FROM studies WHERE start_date IS NOT NULL GROUP BY YEAR(start_date) ORDER BY year

    SELECT YEAR(completion_date) as year, COUNT(*) as completed_count FROM studies WHERE completion_date IS NOT NULL GROUP BY YEAR(completion_date) ORDER BY year

    SELECT YEAR(start_date) as year, AVG(enrollment) as avg_enrollment, COUNT(*) as study_count FROM studies WHERE start_date IS NOT NULL AND enrollment IS NOT NULL GROUP BY YEAR(start_date) ORDER BY year

    SELECT YEAR(start_date) as year, phase, COUNT(*) as count FROM studies WHERE start_date IS NOT NULL AND phase IS NOT NULL GROUP BY YEAR(start_date), phase ORDER BY year, phase

    SELECT YEAR(s.start_date) as year, c.condition_name, COUNT(*) as count FROM studies s JOIN conditions c ON s.study_id = c.study_id WHERE s.start_date IS NOT NULL GROUP BY YEAR(s.start_date), c.condition_name HAVING COUNT(*) >= 3 ORDER BY year, count DESC

Yearly trends (line/area/bar) for starts, completions, enrollment avg, phase evolution, top conditions.​
Search Studies

Query: SELECT * FROM studies WHERE title LIKE %s OR description LIKE %s LIMIT 50

Full-text search on title/description; params prevent injection.​